In [32]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-05 18:35:29--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
65.8.245.178, 65.8.245.171, 65.8.245.51, ...ci6vzurychx.cloudfront.net)... 
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.178|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  19.4MB/s    in 3.5s    

2025-03-05 18:35:33 (17.6 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



# Question 1

In [39]:
pyspark.__version__

'3.3.2'

<b> Answer 1:
'3.3.2' <b>

In [33]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [34]:
df = spark.read.option("header","true").option("inferSchema","true").parquet("yellow_tripdata_2024-10.parquet")
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [35]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-09-30 21:30:44|  2024-09-30 21:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

# Question 2

In [37]:
df = df.repartition(4)
df.write.parquet('yellow_tripdata/2024/10')

<b>Answer 2:
25 MB <b>

In [43]:
df.createOrReplaceTempView('yellow_taxi')

In [61]:
spark.sql("""
SELECT * FROM yellow_taxi;
"""
).count()

3833771

# Question 3

In [50]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [71]:
spark.sql("""
SELECT COUNT(1) FROM yellow_taxi
WHERE DATE(tpep_pickup_datetime) = '2024-10-15';
"""
).show()

+--------+
|count(1)|
+--------+
|  128811|
+--------+



<b> Answer 3: 128811 <b>

# Question 4

In [81]:
spark.sql("""
SELECT MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) FROM yellow_taxi;
"""
).show()

[Stage 139:==========================================>             (9 + 3) / 12]

+--------------------------------------------------------------------------------------------------------------------------------------+
|max(((unix_timestamp(tpep_dropoff_datetime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(tpep_pickup_datetime, yyyy-MM-dd HH:mm:ss)) / 3600))|
+--------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                    162.61777777777777|
+--------------------------------------------------------------------------------------------------------------------------------------+



<b> Answer 4: 162 <b>

# Question 5

URL Example = http://localhost:4040/jobs/

<b> Answer: 4040 <b> 

In [83]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-05 19:27:35--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
65.8.245.51, 65.8.245.171, 65.8.245.50, ...7ci6vzurychx.cloudfront.net)... 
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.01s   

2025-03-05 19:27:36 (1.21 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [87]:

zone_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('taxi_zone_lookup.csv')

In [88]:
zone_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [89]:
zone_df.registerTempTable('zone_lookup')  

# Question 6

In [95]:
spark.sql("""
SELECT zl.Zone FROM zone_lookup AS zl
INNER JOIN yellow_taxi AS yt ON yt.PULocationID = zl.LocationID
GROUP BY 1
ORDER BY COUNT(1) ASC;
""").show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
|       Rikers Island|
|       Arden Heights|
| Green-Wood Cemetery|
|         Jamaica Bay|
|   Rossville/Woodrow|
|Charleston/Totten...|
|       West Brighton|
|Eltingville/Annad...|
|       Port Richmond|
|        Crotona Park|
|         Great Kills|
|     Mariners Harbor|
|Heartland Village...|
|Saint George/New ...|
|             Oakwood|
|New Dorp/Midland ...|
|       Broad Channel|
|         Westerleigh|
|     Pelham Bay Park|
+--------------------+
only showing top 20 rows



<b> Answer: Governor's Island... <b>